# Multi-Objective Molecule Design

Credits to Jin et al 2019 for the development of this algorithm

## Principle

The algorithm is based on the idea of using motifs as building blocks that explain specific properties present in a class of compounds. A property predictor is used to explain the validity of each motif and the most relevant motif is picked out. To generate dual activity i.e compound with 2 different properties, the algorithm combines 2 property-specific motifs based on a maximum common substructure.

The architecture for this algorithm is based on a Variational Autoencoder that utilizes Graph Neural Networks with Message Passing Networks on both the encoder and decoder

The workflow for this algorithm is based as following
1. Building a property predictor
2. Motif extraction 
3. Training a graph neural net on valid molecular structures
4. Generation of molecules based on extracted motifs

## Aim of this Project

We will try to **replicate** the work done by Carlsson and his team. Here is the link to the original paper: https://pubs.acs.org/doi/full/10.1021/acs.jmedchem.8b00204 

Two targets relevant for drug development against Parkinson’s disease, A2AAR and MAO-B, were selected to explore the possibility of discovering dual-target ligand. They used molecular docking screens to evaluate a library of compounds for dual-activity. Instead, we will use this multi-property molecular optimization approach as **an alternative approach** to try generate similar results. 

This sample project will act as a proof of concept study to show that the approach of merging properties specific to different targets can generate real-world molecules that have demonstrated dual-activity. 

# 1. Property Predictor Model

To make it easy for users to follow through, I will only include the workflow for 1 protein target i.e ***A2A Adenosine receptor***. The same workflow was used for generating the model for the 2nd target 

###  Data Preprocessing

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
target = new_client.target
target_query = target.search("A2A Adenosine receptor")
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
selected_target = targets.target_chembl_id = 'CHEMBL251'
activity = new_client.activity
result = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df1 = pd.DataFrame.from_dict(result)
df1

In [ ]:
df1.to_csv("A2AAR_bioactivity_data.csv", index=False)

### Handling missing data

In [ ]:
df1a = df1[df1.standard_value.notna()]
df1a

### Label the compounds as either Active, Inactive or Intermediate

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df1b = df1a[selection]
df1b

In [ ]:
bioactivity_class = []
for i in df1b.standard_value:
    if float(i) <= 1000:
        bioactivity_class.append('active')
    elif float(i) >= 10000:
        bioactivity_class.append('inactive')
    else:
        bioactivity_class.append('intermediate')

In [ ]:
bioactivity_class = pd.Series(bioactivity_class, name='bioactivity_class')
df1c = pd.concat([df1b.reset_index(drop=True), bioactivity_class], axis=1)
df1c

In [ ]:
#Save the data for then next step
df1c.to_csv("A2AAR_preprocessed.csv", index=False)
#df2c.to_csv("MAO-B_preprocessed.csv", index=False)

## Calculating Lipinski descriptors

The Lipinski Rule of 5 is used for describing the ADME profile or the drug-likeness of orally active FDA-approved drugs

The Lipinski's Rule states the following:

    Molecular weight < 500 Dalton
    Octanol-water partition coefficient (LogP) < 5
    Hydrogen bond donors < 5
    Hydrogen bond acceptors < 10

LogP value is more like the solubility of the molecule

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
def lipinski(smiles, verbose=False):
    moldata=[]
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)
    
    basedata = np.arange(1,1)
    i=0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NUmHAcceptors = Lipinski.NumHAcceptors(mol)
        
        row = np.array([desc_MolWt,
                       desc_MolLogP,
                       desc_NumHDonors,
                       desc_NUmHAcceptors])
        if(i == 0):
            basedata = row
        else:
            basedata = np.vstack([basedata,row])
        i = i+1
        
    columnNames = ['MW','LogP','NumHDonors','NumHAcceptors']
    descriptors = pd.DataFrame(data=basedata,columns=columnNames)
    
    return descriptors 

In [ ]:
A2AAR_lipinski = lipinski(df1c.canonical_smiles)
#MAOB_lipinski = lipinski(df2c.canonical_smiles)

In [ ]:
df_A2AAR = pd.concat([df1c, A2AAR_lipinski], axis=1)
#df_MAOB = pd.concat([df2c, MAOB_lipinski], axis=1)

In [ ]:
df_A2AAR

In [ ]:
df_A2AAR.standard_value.describe()

### Convert standard IC50 values to pIC50 values

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []
    
    for i in input['standard_value']:
        molar = float(i) * (10**-9)
        pIC50.append(-np.log10(molar))
        
    input['pIC50'] = pIC50
    X = input.drop('standard_value',1)
    
    return X

In [ ]:
def norm_value(input):
    norm = []
    
    for x in input['standard_value']:
        if x > 100000000:
            x = 100000000
        norm.append(x)
    input['standard_value_norm'] = norm
    X = input.drop('standard_value', 1)
    
    return X

In [ ]:
#Looking at the distribution of the data its not necessary to perform normalization on either A2AAR or the MAOB data
# The data values on  the standard_value column do not exceed 100,000

#A2AAR_norm = norm_value(df_A2AAR)
#A2AAR_norm

In [ ]:
A2AAR_PIC50 = pIC50(df_A2AAR)
A2AAR_PIC50

In [ ]:
A2AAR_PIC50.to_csv("A2AAR_lipinski_pIC50.csv", index=False)

To make the project more fuller, it would be useful to perform ***Exploratory Data Analysis***
on the above data especially on the distribution of the active compounds vs inactives. Such data would help provide the significance of each  Lipinski descriptor

Though for this project, we simply isolate ourselves to the task of using the pIC50 value for building a prediction model

## Calculating Molecular Fingerprints

We will be using the Padel Descriptor Calculator for this part but first we need to extract the canonical-smiles string notation of the molecules

### Let's preprocess the bioactivity data for the SMILES

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
A2AAR_smiles = A2AAR_PIC50[selection]
A2AAR_smiles.to_csv('A2AAR_molecule.smi', sep='\t', index=False, header=False)

In [ ]:
cat A2AAR_molecule.smi | head -3

In [ ]:
cat A2AAR_molecule.smi | wc -l

In [ ]:
#This is a command line version of the Descriptor Calculator that has been configured
!bash padel.sh

The output of the Padel Descriptor Calculator will be a csv file that can be imported into the notebook for further processing

### Generate the X and Y dataframes for model building

In [ ]:
A2AAR_XY = pd.read_csv('ECFP_fingerprints_A2AAR.csv')
A2AAR_XY

In [ ]:
A2AAR_X = A2AAR_XY.drop(columns=['y','w','ids'])
A2AAR_X

In [ ]:
A2AAR_Y = A2AAR_XY['y']
A2AAR_Y

The X dataframe contains the features for model building while the Y dataframe contains the pIC50 value which will be our property of interest in building a regression model

## Building a Regression Model

In [ ]:
# Importing the libraries
import numpy as np # for array operations
import pandas as pd # for working with DataFrames
import requests, io # for HTTP requests and I/O commands
import matplotlib.pyplot as plt # for data visualization
%matplotlib inline

# scikit-learn modules
from sklearn.ensemble import RandomForestRegressor # for building the model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(A2AAR_X, A2AAR_Y, test_size = 0.2, random_state = 28)

In [ ]:
bag_forest_reg = RandomForestRegressor(n_estimators=100,
                                       criterion="mse",
                                       max_features=6,
                                       n_jobs=-1,
                                       random_state=1)

bag_forest_reg.fit(x_train, y_train)

train_pred_y = bag_forest_reg.predict(x_train)
test_pred_y = bag_forest_reg.predict(x_test)

print(f"train_MSE = {mean_squared_error(y_train, train_pred_y)}")
print(f"test_MSE = {mean_squared_error(y_test, test_pred_y)}")

In [ ]:
param_grid = {
    "n_estimators":[100,200,300],
    "max_depth":[10, 50, 100],
    "max_features":[6,8,10,12,14,16]
}

rf_reg = RandomForestRegressor()

rf_reg_tuned = GridSearchCV(estimator=rf_reg,
                            param_grid=param_grid,
                            cv=3,
                            n_jobs=-1,
                            verbose=2)

rf_reg_tuned.fit(x_train, y_train)
grid_best = rf_reg_tuned.best_estimator_

train_pred_y = grid_best.predict(x_train)
test_pred_y = grid_best.predict(x_test)

print(f"train_MSE = {mean_squared_error(y_train, train_pred_y)}")
print(f"test_MSE = {mean_squared_error(y_test, test_pred_y)}")

In this example, I used a random forest regressor to build the prediction model. From the above code cell, I tried doing some small parameter tuning using grid search. You can try to add more parameters for tuning

A suggestion I would have for a higher scoring model would be to try using an LGBM regressor or XgBoost regressor

In [ ]:
# We shall export the better model and pickle it 
import pickle
pickle.dump(grid_best, open('A2AAR_reg.pkl', 'wb'))

# 2. Motif Extraction

Due to the heavy computation of the remaining steps, it is recommended to run the following work on a machine with Nvidia GPU.The python scripts have been implemented to use pythorch's CUDA which utilizes Nvidia GPUs. We will be utilizing python scripts offline which can run from Jupyter notebook

For the motif extraction, we need a set of active compounds for both receptors. We will generate this set of actives from the lipinski dataframe which we saved offline,

In [17]:
import pandas as pd
df_A2AAR = pd.read_csv("A2AAR_lipinski_pIC50.csv")
df_A2AAR

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL280888,CCCSc1c2c(nc(N)n3nc(-c4ccco4)nc23)nn1C,active,329.389,2.35530,1.0,9.0,7.829738
1,CHEMBL22113,COc1ccc(NC(=O)Nc2nc3nn(C)c(SC)c3c3nc(-c4ccco4)...,active,450.484,3.64550,2.0,10.0,7.756962
2,CHEMBL554955,CCNc1c2c(nc(NC(=O)Nc3ccc(OC)cc3)n3nc(-c4ccco4)...,active,483.920,3.77720,3.0,10.0,7.214670
3,CHEMBL21572,CSc1c2c(nc(NC(=O)Cc3ccc4c(c3)OCO4)n3nc(-c4ccco...,active,463.479,2.90280,1.0,11.0,7.667562
4,CHEMBL281129,CSc1c2c(nc(N)n3nc(-c4ccco4)nc23)nn1C,active,301.335,1.57510,1.0,9.0,8.214670
...,...,...,...,...,...,...,...,...
539,CHEMBL2024116,Cc1cc(-c2nnc(N)nc2-c2ccc(F)cc2)cc(C)n1,active,295.321,2.93874,1.0,5.0,9.102373
540,CHEMBL2024116,Cc1cc(-c2nnc(N)nc2-c2ccc(F)cc2)cc(C)n1,active,295.321,2.93874,1.0,5.0,8.000000
541,CHEMBL4797036,Nc1nc(-c2ccc(NC(=O)c3ccc(S(N)(=O)=O)cc3)cc2)cn...,active,501.528,2.02910,3.0,9.0,7.041914
542,CHEMBL4761976,Nc1nc(-c2ccc(NC(=O)CCNC(=O)c3ccc(S(N)(=O)=O)cc...,active,572.607,1.53540,4.0,10.0,6.033389


In [19]:
df_A2AAR_actives = df_A2AAR[df_A2AAR.bioactivity_class == 'active']
df_A2AAR_actives

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL280888,CCCSc1c2c(nc(N)n3nc(-c4ccco4)nc23)nn1C,active,329.389,2.35530,1.0,9.0,7.829738
1,CHEMBL22113,COc1ccc(NC(=O)Nc2nc3nn(C)c(SC)c3c3nc(-c4ccco4)...,active,450.484,3.64550,2.0,10.0,7.756962
2,CHEMBL554955,CCNc1c2c(nc(NC(=O)Nc3ccc(OC)cc3)n3nc(-c4ccco4)...,active,483.920,3.77720,3.0,10.0,7.214670
3,CHEMBL21572,CSc1c2c(nc(NC(=O)Cc3ccc4c(c3)OCO4)n3nc(-c4ccco...,active,463.479,2.90280,1.0,11.0,7.667562
4,CHEMBL281129,CSc1c2c(nc(N)n3nc(-c4ccco4)nc23)nn1C,active,301.335,1.57510,1.0,9.0,8.214670
...,...,...,...,...,...,...,...,...
538,CHEMBL2024116,Cc1cc(-c2nnc(N)nc2-c2ccc(F)cc2)cc(C)n1,active,295.321,2.93874,1.0,5.0,6.844968
539,CHEMBL2024116,Cc1cc(-c2nnc(N)nc2-c2ccc(F)cc2)cc(C)n1,active,295.321,2.93874,1.0,5.0,9.102373
540,CHEMBL2024116,Cc1cc(-c2nnc(N)nc2-c2ccc(F)cc2)cc(C)n1,active,295.321,2.93874,1.0,5.0,8.000000
541,CHEMBL4797036,Nc1nc(-c2ccc(NC(=O)c3ccc(S(N)(=O)=O)cc3)cc2)cn...,active,501.528,2.02910,3.0,9.0,7.041914


In [20]:
selection = ['canonical_smiles']
A2AAR_smiles = df_A2AAR_actives[selection]
A2AAR_smiles.to_csv('A2AAR_actives.txt', sep='\n', index=False, header=False)

In [21]:
## MAOB

In [22]:
import pandas as pd
df_MAOB = pd.read_csv("MAOB_lipinski_pIC50.csv")
df_MAOB

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL156630,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NCc3ccccc3)c(Br)c21,inactive,426.314,4.59450,1.0,4.0,3.000000
1,CHEMBL155754,C/N=C1/CCc2c1n(C)c1ccc(OC(=O)NC)c(Cl)c21,inactive,305.765,2.91500,1.0,4.0,3.000000
2,CHEMBL350093,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,active,335.363,2.69968,0.0,6.0,7.744727
3,CHEMBL161907,O=c1c(=O)c2ccc(OCCCC(F)(F)F)cc2c1=O,active,286.205,1.51730,0.0,4.0,8.045757
4,CHEMBL17079,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=S,active,337.404,4.36527,0.0,6.0,8.356547
...,...,...,...,...,...,...,...,...
4912,CHEMBL4785539,C#CCN(C)CCCCCOc1ccc(C=O)c(OC)c1,intermediate,289.375,2.62180,0.0,4.0,5.241467
4913,CHEMBL4784768,C#CCN(C)CCCCCCOc1ccc(C=O)c(OC)c1,active,303.402,3.01190,0.0,4.0,6.258218
4914,CHEMBL8706,C#CCN(C)CCCOc1ccc(Cl)cc1Cl,inactive,272.175,3.32730,0.0,2.0,4.207538
4915,CHEMBL887,C#CCN[C@@H]1CCc2ccccc21,active,171.243,1.89670,1.0,1.0,6.848630


In [23]:
df_MAOB_actives = df_MAOB[df_MAOB.bioactivity_class == 'active']
df_MAOB_actives

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
2,CHEMBL350093,N#CCCN1CC(=O)OC(c2ccc(OCc3ccccc3)cc2)=N1,active,335.363,2.69968,0.0,6.0,7.744727
3,CHEMBL161907,O=c1c(=O)c2ccc(OCCCC(F)(F)F)cc2c1=O,active,286.205,1.51730,0.0,4.0,8.045757
4,CHEMBL17079,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=S,active,337.404,4.36527,0.0,6.0,8.356547
12,CHEMBL434261,N#CCCn1nnc(-c2ccc(OCc3ccccc3)cc2)n1,active,305.341,2.83278,0.0,6.0,8.698970
16,CHEMBL17092,N#CCCn1nc(-c2ccc(OCc3ccccc3)cc2)oc1=O,active,321.336,2.99598,0.0,6.0,8.657577
...,...,...,...,...,...,...,...,...
4907,CHEMBL4776135,C#CCN(C)CCCCOc1ccc(C(C)=O)cc1,active,259.349,2.61320,0.0,3.0,7.247875
4908,CHEMBL4778143,C#CCN(C)CCCCCCOc1ccc(C(C)=O)cc1,active,287.403,3.39340,0.0,3.0,7.689944
4909,CHEMBL4799286,C#CCN(C)CCCCCOc1cccc(OC)c1,active,261.365,2.80930,0.0,3.0,6.996281
4913,CHEMBL4784768,C#CCN(C)CCCCCCOc1ccc(C=O)c(OC)c1,active,303.402,3.01190,0.0,4.0,6.258218


In [24]:
selection = ['canonical_smiles']
MAOB_smiles = df_MAOB_actives[selection]
MAOB_smiles.to_csv('MAOB_actives.txt', sep='\n', index=False, header=False)

Now that we have the actives for both targets, we can now go ahead with the extraction of the motifs/rationales

In [ ]:
! python mcts.py --data data/jnk3/actives.txt --prop jnk3 --ncpu 4 > jnk3_rationales.txt

In [ ]:
! python mcts.py --data data/gsk3/actives.txt --prop gsk3 --ncpu 4 > gsk3_rationales.txt

To construct multi-property rationales, we can merge the single-property rationales for GSK3 and JNK3: 

In [ ]:
! python merge_rationale.py --rationale1 data/gsk3/rationales.txt --rationale2 data/jnk3/rationales.txt > gsk3_jnk3.txt

# 3. Generative Model Pre-training

The molecule completion model is pre-trained on the ChEMBL dataset. For ease of use, we will use the model generated by the authors of the algorithm stored in the directory ckpt/chembl-h400beta0.3/model.20

# 4. Molecule Generation

A good preprocessing task would be to design dual inhibitors against A2AAR and MAOB with drug-likeness and synthetic accessibility constraints.  This implies finetuning the above model. We have already computed multi-property rationales in data/gsk3_jnk3_qed_sa/rationales.txt. It is a subset of GSK3-JNK3 rationales with QED > 0.6 and SA < 4.0.

In [ ]:
! python finetune.py --init_model ckpt/chembl-h400beta0.3/model.20 --save_dir ckpt/tmp/ --
rationale data/gsk3_jnk3_qed_sa/rationales.txt --num_decode 200 --prop gsk3,jnk3,qed,sa --epoch 30 --alpha 0.5

The molecule generation script will expand the extracted rationales into full molecules. The output is a list of pairs (rationale, molecule), where molecule is the completion of rationale

In [ ]:
! python decode.py --model ckpt/gsk3_jnk3_qed_sa/model.final > outputs.txt

## Evaluation of the Output

We can evaluate the outputs for the four property constraint i.e. gsk3_activity, jnk3_activity, qed, sa by:-

In [ ]:
! python properties.py --prop gsk3,jnk3,qed,sa < outputs.txt | python scripts/qed_sa_dual_eval.py --ref_path data/dual_gsk3_jnk3/actives.txt

The code above includes the reference path to the actives_cpds so as to help compute the novelty score